In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import json
import time

In [4]:
# Укажите путь к вашему драйверу Chromedriver
service = Service()
driver = webdriver.Chrome(service=service)

# Откройте нужный URL
url = "https://www.kitco.com/price/fixes/london-fix"  # Замените на актуальный URL
driver.get(url)

# Дайте время для загрузки страницы (можно использовать WebDriverWait для более надежного решения)
#time.sleep(5)

# Найдите все теги <script> на странице
script_tags = driver.find_elements(By.TAG_NAME, 'script')

# Ищем JSON в скриптах
for script in script_tags:
    if 'application/json' in script.get_attribute('type'):
        try:
            json_data = json.loads(script.get_attribute('innerHTML').strip())
            print(json.dumps(json_data, indent=4))  # Выводим или обрабатываем JSON
        except json.JSONDecodeError:
            print("Не удалось декодировать JSON в скрипте.")

# Закрываем драйвер
driver.quit()

{
    "props": {
        "pageProps": {
            "dehydratedState": {
                "mutations": [],
                "queries": [
                    {
                        "state": {
                            "data": {
                                "GetLondonFixByYearV3": {
                                    "ID": 0,
                                    "currency": "USD",
                                    "results": [
                                        {
                                            "ID": 0,
                                            "goldAM": 2620.55,
                                            "goldPM": 2613.8,
                                            "timestamp": 1734955200,
                                            "silver": 29.605,
                                            "platinumAM": 940,
                                            "platinumPM": 938,
                                            "palladiumAM": 922,
                      

In [28]:
import warnings
import pandas as pd
import httpx  # Используем httpx для асинхронных запросов
import asyncio
from bs4 import BeautifulSoup

warnings.filterwarnings("ignore")

In [ ]:
async def lbma_precious_async():
    url = "https://www.kitco.com/price/fixes/london-fix"

    async def get_raw_data(url):
        async with httpx.AsyncClient() as client:
            response = await client.get(url)  # Асинхронный запрос
            response.raise_for_status()  # Проверка на ошибки

            # Парсинг HTML с помощью BeautifulSoup
            soup = BeautifulSoup(
                response.text, "html.parser"
            )  # Указание парсера, например, 'html.parser'
            
            day = soup.find_all("div", class_="grid")
            
            # Вычленение первичного контента
            day_content = []
            
            for i in range(len(day)):
                day_content.append(day[i].contents)

            # Первичное преобразование
            for container in day_content:
                for element in range(len(container)):
                    try:
                        container[element] = (
                            str(container[element])
                            .replace("<div>", "")
                            .replace("</div>", "")
                            .replace("<!-- -->", "")
                        )
                        container[element]
                    except Exception as e:
                        print(e)

            # Удалим неинформативные строки
            day_content = day_content[1:]
            day_content.pop(1)

            # Преобразование дат и вычлинение значений
            for element in day_content:
                # Преобразованиие даты
                element[0] = pd.to_datetime(element[0]).strftime("%Y-%m-%d")

                # Числовые преобразования
                element[1] = pd.to_numeric(
                    element[1].split("/")[1].strip().replace(",", "")
                )
                element[2] = pd.to_numeric(
                    element[2].split("/")[0].strip().replace(",", "")
                )
                element[3] = pd.to_numeric(
                    element[3].split("/")[1].strip().replace(",", "")
                )
                element[4] = pd.to_numeric(
                    element[4].split("/")[1].strip().replace(",", "")
                )

            # Формируем датафрейм
            day_content = pd.DataFrame(
                day_content, columns=["Date", "Gold", "Silver", "Platinum", "Palladium"]
            )

            # Преобразуем типы
            day_content["Date"] = pd.to_datetime(day_content["Date"])
            day_content = day_content.sort_values("Date").reset_index(drop=True)

            # Возьмем последние 5 строк
            day_update = day_content.tail()
            
            # Склеим с историческими данными
            historical_df = pd.read_excel("./data/kitko_db.xlsx", index_col=0)
            historical_df = (
                pd.concat([historical_df, day_update], axis=0)
                .reset_index(drop=True)
                .drop_duplicates()
            )

            with pd.ExcelWriter(
                        "./data/kitko_db.xlsx",
                        date_format="YYYY-MM-DD",
                        datetime_format="YYYY-MM-DD",
                    ) as writer:
                        historical_df.to_excel(writer, sheet_name="kitco_metall")

            print("KITKO_main is done!!!")
            # display(historical_df.tail(10))

            return historical_df

    try:
        # Получаем данные асинхронно
        gold = await get_raw_data(url)
        display(gold)

    except Exception as error:
        print("Произошла ошибка:", error)


# Запускаем асинхронную функцию
await lbma_precious_async()  # Просто вызываем, без asyncio.run()

KITKO_main is done!!!


,Date,Gold,Silver,Platinum,Palladium
0,2023-01-03,1843.25,24.295,1082.0,1795.0
1,2023-01-04,1857.30,24.290,1080.0,1736.0
2,2023-01-05,1834.00,23.410,1062.0,1783.0
3,2023-01-06,1852.20,23.455,1073.0,1784.0
4,2023-01-09,1878.85,23.850,1092.0,1793.0
...,...,...,...,...,...
497,2024-12-17,2636.35,30.310,932.0,934.0
498,2024-12-18,2635.65,30.360,930.0,929.0
499,2024-12-19,2592.45,29.490,929.0,909.0
500,2024-12-20,2616.45,28.800,930.0,921.0


---
### LME

In [43]:
async def lme_async():
    url = "https://www.lme.com/Metals/Non-ferrous#tabIndex=1"

    async def get_raw_data(url):
        async with httpx.AsyncClient() as client:
            response = await client.get(url)  # Асинхронный запрос
            response.raise_for_status()  # Проверка на ошибки

            # Парсинг HTML с помощью BeautifulSoup
            soup = BeautifulSoup(
                response.text, "html.parser"
            )  # Указание парсера, например, 'html.parser'
            
            # Вытаскиваем данные из табличного блока
            data_raw = soup.find_all("div", class_="metal-block-row__blocks")

            # Приводим к тексту
            metalls_raw = data_raw[0].text

            # Убираем все лишнее
            metalls_raw = metalls_raw.replace(" ", "").replace("LME", "LME_").split("\n")
            metalls_raw = " ".join(metalls_raw).strip().split(" ")

            # Конфигурируем новый лист по металлам
            metalls_raw = [metalls_raw[i : i + 4] for i in range(0, len(metalls_raw), 4)]

            # Обрезаем до нужной инфы
            metalls_raw = [i[:2] for i in metalls_raw]

            # Приведем цены к числовому формату
            for metall in metalls_raw:
                metall[1] = pd.to_numeric(metall[1])

            # Формируем строку датафрейма по металлам
            dict_list = [{item[0]: item[1]} for item in metalls_raw]

            metall_df = pd.DataFrame()

            for i in dict_list:
                keys = list(i.keys())
                values = list(i.values())
                metall_df[keys[0]] = values

            # Дропаем ненужные колонки
            metall_df.drop(columns=["LME_AluminiumAlloy", "LME_NASAAC"], inplace=True)
            metall_df = metall_df.rename(
                columns={
                    "LME_Aluminium": "aluminium",
                    "LME_Copper": "copper",
                    "LME_Lead": "lead",
                    "LME_Nickel": "nickel",
                    "LME_Zinc": "zink",
                    "LME_Tin": "tin",
                }
            )

            # Достанем в вставим дату
            date_raw = soup.find_all("span", class_="metal-block-container__refreshed-on")
            date_raw = date_raw[0]
            date_raw = str(date_raw).split(">")[1]
            date_raw = date_raw.replace("\xa0", "").replace("</span", "")
            date_raw = pd.to_datetime(date_raw) - pd.Timedelta(days=1)

            metall_df["date"] = date_raw

            # Поменяем порядок колонок
            new_column_order = [
                "date",
                "aluminium",
                "copper",
                "lead",
                "nickel",
                "zink",
                "tin",
            ]
            metall_df = metall_df[new_column_order]

            # Сохраняем полученные результаты
            lme_db = pd.read_excel("./data/LME_db_new.xlsx", index_col=0)
            lme_db = pd.concat([lme_db, metall_df]).reset_index(drop=True)

            # Дропаем дубликаты
            lme_db = lme_db.drop_duplicates(subset=lme_db.columns.to_list()[1:])

            

            return lme_db

    try:
        # Получаем данные асинхронно
        lme_db = await get_raw_data(url)
        display(lme_db)

    except Exception as error:
        print("Произошла ошибка:", error)


# Запускаем асинхронную функцию
await lme_async()  # Просто вызываем, без asyncio.run()

Произошла ошибка: Client error '403 Forbidden' for url 'https://www.lme.com/Metals/Non-ferrous#tabIndex=1'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403


In [45]:
import cloudscraper  # Импортируем cloudscraper
from bs4 import BeautifulSoup  # Импортируем BeautifulSoup
import pandas as pd
import asyncio

async def lme_async():
    url = "https://www.lme.com/Metals/Non-ferrous#tabIndex=1"

    # Создайте экземпляр cloudscraper
    scraper = cloudscraper.create_scraper()

    async def get_raw_data(url):
        # Используем cloudscraper для получения данных
        response = scraper.get(url)  # Синхронный запрос
        response.raise_for_status()  # Проверка на ошибки

        # Парсинг HTML с помощью BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")  # Указание парсера

        # Вытаскиваем данные из табличного блока
        data_raw = soup.find_all("div", class_="metal-block-row__blocks")

        # Приводим к тексту
        if not data_raw:
            raise ValueError("Не удалось найти данные с указанным классом.")
        metalls_raw = data_raw[0].text

        # Убираем лишнее
        metalls_raw = metalls_raw.replace(" ", "").replace("LME", "LME_").split("\n")
        metalls_raw = " ".join(metalls_raw).strip().split(" ")

        # Конфигурируем новый лист по металлам
        metalls_raw = [metalls_raw[i : i + 4] for i in range(0, len(metalls_raw), 4)]

        # Обрезаем до нужной инфы
        metalls_raw = [i[:2] for i in metalls_raw]

        # Приводим цены к числовому формату
        for metall in metalls_raw:
            metall[1] = pd.to_numeric(metall[1])

        # Формируем строку датафрейма по металлам
        dict_list = [{item[0]: item[1]} for item in metalls_raw]

        metall_df = pd.DataFrame()

        for i in dict_list:
            keys = list(i.keys())
            values = list(i.values())
            metall_df[keys[0]] = values

        # Дропаем ненужные колонки
        metall_df.drop(columns=["LME_AluminiumAlloy", "LME_NASAAC"], inplace=True)
        metall_df = metall_df.rename(
            columns={
                "LME_Aluminium": "aluminium",
                "LME_Copper": "copper",
                "LME_Lead": "lead",
                "LME_Nickel": "nickel",
                "LME_Zinc": "zink",
                "LME_Tin": "tin",
            }
        )

        # Достаем и вставим дату
        date_raw = soup.find_all("span", class_="metal-block-container__refreshed-on")
        date_raw = date_raw[0]
        date_raw = str(date_raw).split(">")[1]
        date_raw = date_raw.replace("\xa0", "").replace("</span", "")
        date_raw = pd.to_datetime(date_raw) - pd.Timedelta(days=1)

        metall_df["date"] = date_raw

        # Поменяем порядок колонок
        new_column_order = [
            "date",
            "aluminium",
            "copper",
            "lead",
            "nickel",
            "zink",
            "tin",
        ]
        metall_df = metall_df[new_column_order]

        # Сохраняем результаты
        lme_db = pd.read_excel("./data/LME_db_new.xlsx", index_col=0)
        lme_db = pd.concat([lme_db, metall_df]).reset_index(drop=True)

        # Дропаем дубликаты
        lme_db = lme_db.drop_duplicates(subset=lme_db.columns.to_list()[1:])

        return lme_db

    try:
        # Получаем данные асинхронно
        lme_db = await get_raw_data(url)
        print(lme_db)

    except Exception as error:
        print("Произошла ошибка:", error)

# Запускаем асинхронную функцию
await lme_async()  # Просто вызываем, без asyncio.run()

Произошла ошибка: 403 Client Error: Forbidden for url: https://www.lme.com/Metals/Non-ferrous#tabIndex=1


____

In [53]:
import warnings
import pandas as pd
import httpx  # Используем httpx для асинхронных запросов
import asyncio

warnings.filterwarnings("ignore")

################################################################
#########################   LBMA (KITCO subs)-new ##################
################################################################


async def lbma_prescious_async():
    url_gold = "https://prices.lbma.org.uk/json/gold_pm.json"
    url_silver = "https://prices.lbma.org.uk/json/silver.json"
    url_platinum = "https://prices.lbma.org.uk/json/platinum_pm.json"
    url_paladium = "https://prices.lbma.org.uk/json/palladium_pm.json"

    async def get_raw_data(url, metall="Default"):
        async with httpx.AsyncClient() as client:
            response = await client.get(url)  # Асинхронный запрос
            response.raise_for_status()  # Проверка на ошибки

            raw_data = pd.read_json(response.text)
            data = raw_data[["d", "v"]]
            data["v"] = data["v"].apply(lambda x: x[0])
            data["d"] = pd.to_datetime(data["d"])

            data.rename(columns={"d": "Date", "v": metall}, inplace=True)
            return data.tail(4)

    try:
        # Получаем данные асинхронно
        gold = await get_raw_data(url_gold, metall="Gold")
        silver = await get_raw_data(url_silver, metall="Silver")
        platinum = await get_raw_data(url_platinum, metall="Platinum")
        paladium = await get_raw_data(url_paladium, metall="Palladium")

        # Теперь это DataFrame
        result_df = (
            gold.merge(silver, on="Date", how="outer")
            .merge(platinum, on="Date", how="outer")
            .merge(paladium, on="Date", how="outer")
        )

        result_df.fillna(value=0, inplace=True)
        result_df = result_df.sort_values(by="Date")
        result_df.reset_index(inplace=True, drop=True)

        historical = pd.read_excel("./data/lbme_kitco_subs.xlsx", index_col=0)

        result = pd.concat([historical, result_df], axis=0).reset_index(drop=True)

        result.drop_duplicates(inplace=True)

        result = result.sort_values(by="Date")

        result.to_excel("./data/lbme_kitco_subs.xlsx", sheet_name="lbme_metall")
        display(result)
        print("LBMA is done!!!")

    except Exception as error:
        print("Произошла ошибка:", error)

# Пример запуска асинхронной функции
await lbma_prescious_async()


,Date,Gold,Silver,Platinum,Palladium
0,1968-01-02,0.00,2.173,0.0,0.0
1,1968-01-03,0.00,2.225,0.0,0.0
2,1968-01-04,0.00,2.171,0.0,0.0
3,1968-01-05,0.00,2.157,0.0,0.0
4,1968-01-08,0.00,2.163,0.0,0.0
...,...,...,...,...,...
14422,2024-12-17,2636.35,30.310,932.0,934.0
14423,2024-12-18,2635.65,30.355,930.0,929.0
14424,2024-12-19,2592.45,29.490,929.0,909.0
14425,2024-12-20,2616.45,28.795,930.0,921.0


LBMA is done!!!


In [52]:
import cfscrape
################################################################
#########################   LBMA (KITCO subs) ##################
################################################################


async def lbma_prescious_async():
    url_gold = "https://prices.lbma.org.uk/json/gold_pm.json"
    url_silver = "https://prices.lbma.org.uk/json/silver.json"
    url_platinum = "https://prices.lbma.org.uk/json/platinum_pm.json"
    url_paladium = "https://prices.lbma.org.uk/json/palladium_pm.json"

    async def get_raw_data(url, metall="Default"):
        scraper = cfscrape.create_scraper()
        scraped_data = scraper.get(url)

        # scraped_data = get_session(url=url)

        raw_data = pd.read_json(scraped_data.text)
        data = raw_data[["d", "v"]]
        data["v"] = data["v"].apply(lambda x: x[0])
        data["d"] = pd.to_datetime(data["d"])

        data.rename(columns={"d": "Date", "v": metall}, inplace=True)
        data = data.tail(4)

        return data

    try:
        gold = await get_raw_data(url_gold, metall="Gold")
        silver = await get_raw_data(url_silver, metall="Silver")
        platinum = await get_raw_data(url_platinum, metall="Platinum")
        paladium = await get_raw_data(url_paladium, metall="Palladium")

        result_df = (
            gold.merge(silver, on="Date", how="outer")
            .merge(platinum, on="Date", how="outer")
            .merge(paladium, on="Date", how="outer")
        )
        

        result_df.fillna(value=0, inplace=True)
        result_df = result_df.sort_values(by="Date")
        result_df.reset_index(inplace=True, drop=True)
        
        display(result_df)
        
        historical = pd.read_excel("./data/lbme_kitco_subs.xlsx", index_col=0)

        result = pd.concat([historical, result_df], axis=0).reset_index(drop=True)

        result.drop_duplicates(inplace=True)

        result = result.sort_values(by="Date")

        result.to_excel("./data/lbme_kitco_subs.xlsx", sheet_name="lbme_metall")

        print("LBMA is done!!!")
        display(result.tail(10))

    except Exception as error:
        print(error)
        
        
    await lbma_prescious_async()

---

In [54]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options  # Импортируем Options
import time
import pandas as pd
from bs4 import BeautifulSoup

################################################################
##########################   LME ###############################
################################################################

# Асинхронная функция
async def lme_selenium_async():
    url = "https://www.lme.com/Metals/Non-ferrous#tabIndex=1"
    
    # Настраиваем опции хрома
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Включаем безголовый режим
    chrome_options.add_argument("--no-sandbox")  # Для Linux
    chrome_options.add_argument("--disable-dev-shm-usage")  # Для Linux
    chrome_options.add_argument("--window-size=1920x1080")  # Задаем размер окна
    chrome_options.add_argument("--disable-gpu")  # Отключаем аппаратное ускорение

    service = Service()  # Убедитесь, что у вас есть путь к драйверу, если он не установлен в PATH
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        driver.get(url=url)

        time.sleep(2)

        driver.execute_script(
            "window.scrollTo(0, window.scrollY + window.innerHeight);"
        )
        driver.execute_script(
            "window.scrollTo(0, window.scrollY + window.innerHeight);"
        )

        time.sleep(2)

        html_code = driver.page_source

        soup = BeautifulSoup(html_code, "html.parser")
        # Вытаскиваем данные из табличного блока
        data_raw = soup.find_all("div", class_="metal-block-row__blocks")

        # Приводим к тексту
        metalls_raw = data_raw[0].text

        # Убираем все лишнее
        metalls_raw = metalls_raw.replace(" ", "").replace("LME", "LME_").split("\n")
        metalls_raw = " ".join(metalls_raw).strip().split(" ")

        # Конфигурируем новый лист по металлам
        metalls_raw = [metalls_raw[i : i + 4] for i in range(0, len(metalls_raw), 4)]

        # Обрезаем до нужной инфы
        metalls_raw = [i[:2] for i in metalls_raw]

        # Приведем цены к числовому формату
        for metall in metalls_raw:
            metall[1] = pd.to_numeric(metall[1])

        # Формируем строку датафрейма по металлам
        dict_list = [{item[0]: item[1]} for item in metalls_raw]

        metall_df = pd.DataFrame()

        for i in dict_list:
            keys = list(i.keys())
            values = list(i.values())
            metall_df[keys[0]] = values

        # Дропаем ненужные колонки
        metall_df.drop(columns=["LME_AluminiumAlloy", "LME_NASAAC"], inplace=True)
        metall_df = metall_df.rename(
            columns={
                "LME_Aluminium": "aluminium",
                "LME_Copper": "copper",
                "LME_Lead": "lead",
                "LME_Nickel": "nickel",
                "LME_Zinc": "zink",
                "LME_Tin": "tin",
            }
        )

        # Достанем и вставим дату
        date_raw = soup.find_all("span", class_="metal-block-container__refreshed-on")
        date_raw = date_raw[0]
        date_raw = str(date_raw).split(">")[1]
        date_raw = date_raw.replace("\xa0", "").replace("</span", "")
        date_raw = pd.to_datetime(date_raw) - pd.Timedelta(days=1)

        metall_df["date"] = date_raw

        # Поменяем порядок колонок
        new_column_order = [
            "date",
            "aluminium",
            "copper",
            "lead",
            "nickel",
            "zink",
            "tin",
        ]
        metall_df = metall_df[new_column_order]

        # Сохраняем полученные результаты
        lme_db = pd.read_excel("./data/LME_db_new.xlsx", index_col=0)
        lme_db = pd.concat([lme_db, metall_df]).reset_index(drop=True)

        # Дропаем дубликаты
        lme_db = lme_db.drop_duplicates(subset=lme_db.columns.to_list()[1:])

        # Сохраняем
        with pd.ExcelWriter(
            "./data/LME_db_new.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            lme_db.to_excel(writer, sheet_name="LME_main")

        print("LME_main is done!!!")

    except Exception as error:
        print("Произошла ошибка:", error)

    finally:
        driver.quit()  # Закрываем драйвер без необходимости явно закрывать окно

await lme_selenium_async()

Произошла ошибка: list index out of range


In [60]:
import asyncio
################################################################
##########################   LME ###############################
################################################################
# Асинхронная функция
async def lme_selenium_async():
    url = "https://www.lme.com/Metals/Non-ferrous#tabIndex=1"
    
    # Настраиваем опции хрома
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Включаем безголовый режим
    #chrome_options.add_argument("--no-sandbox")  # Для Linux
    #chrome_options.add_argument("--disable-dev-shm-usage")  # Для Linux
    chrome_options.add_argument("--window-size=1920x1080")  # Задаем размер окна
    chrome_options.add_argument("--disable-gpu")  # Отключаем аппаратное ускорение

    service = Service()  # Убедитесь, что у вас есть путь к драйверу, если он не установлен в PATH
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        # driver.maximize_window()
        driver.get(url=url)

        time.sleep(5)

        driver.execute_script(
            "window.scrollTo(0, window.scrollY + window.innerHeight);"
        )
        driver.execute_script(
            "window.scrollTo(0, window.scrollY + window.innerHeight);"
        )

        time.sleep(5)

        html_code = driver.page_source
        #driver.close()

        soup = BeautifulSoup(html_code, "html.parser")
        # Вытаскиваем данные из табличного блока
        data_raw = soup.find_all("div", class_="metal-block-row__blocks")
        
        display(data_raw)

        # Приводим к тексту
        metalls_raw = data_raw[0].text

        # Убираем все лишнее
        metalls_raw = metalls_raw.replace(" ", "").replace("LME", "LME_").split("\n")
        metalls_raw = " ".join(metalls_raw).strip().split(" ")

        # Конфигурируем новый лист по металлам
        metalls_raw = [metalls_raw[i : i + 4] for i in range(0, len(metalls_raw), 4)]

        # Обрезаем до нужной инфы
        metalls_raw = [i[:2] for i in metalls_raw]

        # Приведем цены к числовому формату
        for metall in metalls_raw:
            metall[1] = pd.to_numeric(metall[1])

        # Формируем строку датафрейма по металлам
        dict_list = [{item[0]: item[1]} for item in metalls_raw]

        metall_df = pd.DataFrame()

        for i in dict_list:
            keys = list(i.keys())
            values = list(i.values())
            metall_df[keys[0]] = values

        # Дропаем ненужные колонки
        metall_df.drop(columns=["LME_AluminiumAlloy", "LME_NASAAC"], inplace=True)
        metall_df = metall_df.rename(
            columns={
                "LME_Aluminium": "aluminium",
                "LME_Copper": "copper",
                "LME_Lead": "lead",
                "LME_Nickel": "nickel",
                "LME_Zinc": "zink",
                "LME_Tin": "tin",
            }
        )

        # Достанем в вставим дату
        date_raw = soup.find_all("span", class_="metal-block-container__refreshed-on")
        date_raw = date_raw[0]
        date_raw = str(date_raw).split(">")[1]
        date_raw = date_raw.replace("\xa0", "").replace("</span", "")
        date_raw = pd.to_datetime(date_raw) - pd.Timedelta(days=1)

        metall_df["date"] = date_raw

        # Поменяем порядок колонок
        new_column_order = [
            "date",
            "aluminium",
            "copper",
            "lead",
            "nickel",
            "zink",
            "tin",
        ]
        metall_df = metall_df[new_column_order]
        
        display(metall_df)
        # Сохраняем полученные результаты
        lme_db = pd.read_excel("./data/LME_db_new.xlsx", index_col=0)
        lme_db = pd.concat([lme_db, metall_df]).reset_index(drop=True)
        

        # Дропаем дубликаты
        lme_db = lme_db.drop_duplicates(subset=lme_db.columns.to_list()[1:])

        # Сохраняем
        with pd.ExcelWriter(
            "./data/LME_db_new.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            lme_db.to_excel(writer, sheet_name="LME_main")
        
        print("LME_main is done!!!")
        # display(lme_db.tail(10))

    except Exception as error:
        print(error)

    finally:
        driver.close()
        driver.quit()
        
await lme_selenium_async()


[]

list index out of range


In [63]:
import cloudscraper  # Импортируем cloudscraper
from bs4 import BeautifulSoup  # Импортируем BeautifulSoup

# URL, который нужно открыть
url = "https://www.lme.com/api/trading-data/metal-block?datasourceIds=1b72897c-6a30-480c-a74a-cfc2faaaff16&datasourceIds=fc09fcde-6438-4834-9221-98da9ed54eea&datasourceIds=b411edcc-fc99-4da8-ba18-da82b8a5dad4&datasourceIds=6326cb5f-0770-4ba6-a3b4-871ecf0566f7&datasourceIds=14f25c1d-fd36-459d-a228-e088180bfb75&datasourceIds=1dcd0785-8af8-4df3-aae0-084e9a686654&datasourceIds=4b9fe162-6e23-4566-ba72-6f798c10c605&datasourceIds=e7c48262-7026-401e-8e12-3311783bf629"

# Создаем экземпляр cloudscraper
scraper = cloudscraper.create_scraper()

try:
    # Выполняем GET-запрос
    response = scraper.get(url)
    response.raise_for_status()  # Проверка на ошибки

    # Получаем HTML-код страницы
    html_code = response.text

    # Парсинг HTML с помощью BeautifulSoup
    soup = BeautifulSoup(html_code, "html.parser")

    # Пример: выводим заголовки страницы
    print(soup.title.string)

    # Дополнительно: можно извлекать данные, как вам нужно
    # Например, получать все ссылки на странице
    links = soup.find_all("a")
    for link in links:
        print(link.get("href"))

except Exception as error:
    print("Произошла ошибка:", error)

Произошла ошибка: 403 Client Error: Forbidden for url: https://www.lme.com/api/trading-data/metal-block?datasourceIds=1b72897c-6a30-480c-a74a-cfc2faaaff16&datasourceIds=fc09fcde-6438-4834-9221-98da9ed54eea&datasourceIds=b411edcc-fc99-4da8-ba18-da82b8a5dad4&datasourceIds=6326cb5f-0770-4ba6-a3b4-871ecf0566f7&datasourceIds=14f25c1d-fd36-459d-a228-e088180bfb75&datasourceIds=1dcd0785-8af8-4df3-aae0-084e9a686654&datasourceIds=4b9fe162-6e23-4566-ba72-6f798c10c605&datasourceIds=e7c48262-7026-401e-8e12-3311783bf629


In [64]:
import cloudscraper  # Импортируем cloudscraper
from bs4 import BeautifulSoup  # Импортируем BeautifulSoup

# URL, который нужно открыть
url = "https://www.lme.com/api/trading-data/metal-block?datasourceIds=1b72897c-6a30-480c-a74a-cfc2faaaff16&datasourceIds=fc09fcde-6438-4834-9221-98da9ed54eea&datasourceIds=b411edcc-fc99-4da8-ba18-da82b8a5dad4&datasourceIds=6326cb5f-0770-4ba6-a3b4-871ecf0566f7&datasourceIds=14f25c1d-fd36-459d-a228-e088180bfb75&datasourceIds=1dcd0785-8af8-4df3-aae0-084e9a686654&datasourceIds=4b9fe162-6e23-4566-ba72-6f798c10c605&datasourceIds=e7c48262-7026-401e-8e12-3311783bf629"

# Создаем экземпляр cloudscraper
scraper = cloudscraper.create_scraper()

# Добавляем необходимые заголовки
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "application/json",  # Задаем тип содержимого, который ожидаем
    "Content-Type": "application/json"
}

try:
    # Выполняем GET-запрос с заголовками
    response = scraper.get(url, headers=headers)
    response.raise_for_status()  # Проверка на ошибки

    # Получаем HTML-код страницы или JSON в зависимости от ответа
    json_data = response.json()  # Парсинг JSON-ответа
    print(json_data)  # Выводим полученные данные

except Exception as error:
    print("Произошла ошибка:", error)

Произошла ошибка: 403 Client Error: Forbidden for url: https://www.lme.com/api/trading-data/metal-block?datasourceIds=1b72897c-6a30-480c-a74a-cfc2faaaff16&datasourceIds=fc09fcde-6438-4834-9221-98da9ed54eea&datasourceIds=b411edcc-fc99-4da8-ba18-da82b8a5dad4&datasourceIds=6326cb5f-0770-4ba6-a3b4-871ecf0566f7&datasourceIds=14f25c1d-fd36-459d-a228-e088180bfb75&datasourceIds=1dcd0785-8af8-4df3-aae0-084e9a686654&datasourceIds=4b9fe162-6e23-4566-ba72-6f798c10c605&datasourceIds=e7c48262-7026-401e-8e12-3311783bf629


In [72]:
import asyncio
from playwright.async_api import async_playwright  # Импортируем асинхронный интерфейс Playwright

async def main_lme():
    async with async_playwright() as p:
        # Запускаем браузер в безголовом режиме
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # Открываем нужный URL
        url = "https://www.lme.com/Metals/Non-ferrous#tabIndex=1"
        await page.goto(url)

        # Ждем, пока данные загрузятся (можно использовать более надежные методы ожидания)
        await page.wait_for_timeout(2000)  # Ждем 2 секунды
        await page.evaluate("window.scrollTo(0, window.innerHeight);")  # Скроллим страницу вниз
        await page.wait_for_timeout(2000)

        # Получаем HTML-код страницы
        html_code = await page.content()

        # Парсим данные с помощью BeautifulSoup
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(html_code, "html.parser")

        # Вытаскиваем данные из табличного блока
        data_raw = soup.find_all("div", class_="metal-block-row__blocks")

        if data_raw:
            # Приводим к тексту
            metalls_raw = data_raw[0].text.strip()

            # Обрабатываем текст
            metalls_raw = metalls_raw.replace(" ", "").replace("LME", "LME_").split("\n")
            metalls_raw = " ".join(metalls_raw).strip().split(" ")
            metalls_raw = [metalls_raw[i : i + 4] for i in range(0, len(metalls_raw), 4)]
            metalls_raw = [i[:2] for i in metalls_raw]
            print(metalls_raw)  # Выводим результат

        else:
            print("Данные не найдены.")

        # Закрываем браузер
        
await main_lme()

Error: BrowserType.launch: Executable doesn't exist at /Users/kirillkuznecov/Library/Caches/ms-playwright/chromium_headless_shell-1148/chrome-mac/headless_shell
╔════════════════════════════════════════════════════════════╗
║ Looks like Playwright was just installed or updated.       ║
║ Please run the following command to download new browsers: ║
║                                                            ║
║     playwright install                                     ║
║                                                            ║
║ <3 Playwright Team                                         ║
╚════════════════════════════════════════════════════════════╝

In [69]:
!/usr/local/bin/python3.11 -m pip install playwright


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 379.1 kB/s eta 0:00:0000:0100:03


In [74]:
import yadisk

In [75]:
token = pd.read_csv("./misc/token.csv", header=None)
token = token.iloc[0, 0]

y = yadisk.YaDisk(token=token)

In [93]:
y.open_file("macro_db")

FileNotFoundError: [Errno 2] No such file or directory: 'macro_db'

In [100]:
pd.read_excel("./data/LME_db_new.xlsx", index_col=0).to_csv("./data/LME_db_new.csv", sep=",", index=False)